In [1]:
import numpy as np
from modules import RandomForestClustRegressor, RotationLSH

%load_ext autoreload
%autoreload 2

In [2]:
X = np.random.randn(1000, 486)
Y = np.random.randn(1000, 81)

lsh = RotationLSH()
X_comp = lsh.fit_transform(X, n_iter=10)

rf = RandomForestClustRegressor()
rf.fit(X, X_comp, Y)

RandomForestClustRegressor(bootstrap=True, max_depth=15, max_features=1,
              max_leaf_nodes=None, min_samples_leaf=64,
              min_samples_split=2, n_estimators=10, n_jobs=None,
              random_state=7, warm_start=False)

In [3]:
X_test = np.random.randn(10,486)
X_test_comp = lsh.transform(X)

rf.predict(X_test, X_test_comp).shape

(10, 81)